# Training Drive DATASET

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from AdnGAN import Generator, Discriminator,tools

## Training

### Unchangeable (libraries, transofrmations, hyperparameters)

In [ ]:
# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 64
IMAGE_SIZE = 64
CHANNELS_IMG = 1 # change to 1 for grayscale images
NOISE_DIM = 100
NUM_EPOCHS = 50
FEATURES_DISC = 64
FEATURES_GEN = 64

gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN,IMAGE_SIZE).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC,IMAGE_SIZE).to(device)
tools.initialize_weights(gen)
tools.initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

gen.train()
disc.train()

print(".")

### Dataset

In [3]:
# Transofrmations
transform = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=CHANNELS_IMG),
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
    ]
)

In [4]:
dataset = ImageFolder(root="../Drive_data/seg")

In [5]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_folder = ImageFolder(root=root_dir,transform=transform)
        self.transform = transform

    def __len__(self):
        return max(len(self.image_folder),BATCH_SIZE)

    def __getitem__(self, idx):
        idx = idx % len(self.image_folder)
        image, label = self.image_folder[idx]
        return image
    

custom_dataset = CustomDataset(root_dir="../Drive_data/seg", transform=transform)
dataloader = DataLoader(custom_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
instance = next(iter(dataloader))
img = instance[0].cpu().detach().numpy().transpose(1, 2, 0)
print(len(img))

import matplotlib.pyplot as plt

def visualize_bw_image(image):
    # Reshape the image to remove the singleton dimension if present
    if len(image.shape) == 3 and image.shape[0] == 1:
        image = image.squeeze()

    # Display the image
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()


"""

# Assuming `image` is your black and white image of shape (1, 128, 128)
for image in instance:
    img = image.cpu().detach().numpy().transpose(1, 2, 0)
    visualize_bw_image(img)
    """

64


'\n\n# Assuming `image` is your black and white image of shape (1, 128, 128)\nfor image in instance:\n    img = image.cpu().detach().numpy().transpose(1, 2, 0)\n    visualize_bw_image(img)\n    '

### Training looop

In [8]:
for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    for batch_idx, real in enumerate(dataloader):
        real = real.to(torch.float32).to(device)
        noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        #Visualisation
        if epoch % 5 == 0:
            noise = torch.randn(1, NOISE_DIM, 1, 1).to(device)
            g = gen(noise)
            pic = g[0].cpu().detach().numpy().transpose(1, 2, 0)
            visualize_bw_image(pic)


        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            """
            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1
            """

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Did i generated some good shit ?

In [ ]:
noise = torch.randn(1, NOISE_DIM, 1, 1).to(device)
g = gen(noise)
print(g.shape)
pic = g[0].cpu().detach().numpy().transpose(1, 2, 0)

visualize_bw_image(pic)


In [ ]:
print("Hello")

In [ ]:
torch.save(gen.state_dict(), ".\gen_128.pth")